## Image Classifier using CNN

This project shows how to build an image classifier of your choice based on dataset of images you provide.
<br>We will use Convolutional Neural Networks (CNN) this time

In [20]:
#install required libraries to draw model architecture
#!conda install -c anaconda pydot -y
!conda install -c anaconda graphviz -y

Solving environment: done

## Package Plan ##

  environment location: /anaconda3

  added / updated specs: 
    - pydot


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    graphviz-2.40.1            |       hefbbd9a_2         6.7 MB  anaconda
    ca-certificates-2018.03.07 |                0         124 KB  anaconda
    pydot-1.3.0                |           py27_1          39 KB  anaconda
    fribidi-1.0.5              |       h1de35cc_0          62 KB  anaconda
    conda-4.5.11               |           py27_0         1.0 MB  anaconda
    pango-1.42.4               |       h060686c_0         523 KB  anaconda
    certifi-2018.10.15         |           py27_0         138 KB  anaconda
    ------------------------------------------------------------
                                           Total:         8.6 MB

The following NEW packages will be INSTALLED:

    fribidi:         1.0.5-h1d

This function, __load_dataset_from_folder__, will load image files from specific path. It will return an array of samples with their assigned label, and a list of labels.
<br>An example of dataset folder structure is the following:
<br>__Shoes__
<br>|-__children__
<br>|----|- imgA.jpg
<br>|----|- imgB.jpg
<br>|----| ....
<br>|----|- imgN.jpg
<br>|-__men__
<br>|----|- imgA.jpg
<br>|----|- imgB.jpg
<br>|----| ....
<br>|----|- imgN.jpg
<br>|-__women__
<br>|----|- imgA.jpg
<br>|----|- imgB.jpg
<br>|----| ....
<br>|----|- imgN.jpg

This will generate a dataset for shoes with labels: 
<br>children: 0
<br>men     :1
<br>women :2

In [ ]:
import numpy as np
import PML

TARGET_WIDTH=150
TARGET_HEIGHT=150

print("Loading Training Samples...")
samples,labels=PML.load_img_dataset_from_folder("datasets/catsdogs",target_size=(TARGET_WIDTH,TARGET_HEIGHT))

for i,l in enumerate(labels):
    samples_count=len([s for s in samples if s[1]==i])
    print("{0}- [{1}] with total of {2} samples".format(i,l,samples_count))
    

#load test samples
print("Loading Test Samples...")
sample_test,labels_test=PML.load_img_dataset_from_folder("test/catsdogs/",target_size=(TARGET_WIDTH,TARGET_HEIGHT))
for i,l in enumerate(labels_test):
    samples_count=len([s for s in sample_test if s[1]==i])
    print("{0}- [{1}] with total of {2} samples".format(i,l,samples_count))
    

Show some random images from the loaded dataset

In [ ]:

import PML
%matplotlib inline
PML.plot_random_images(samples,labels=labels,rows=3,cols=5)

## Preprocessing

Reshape the loaded samples to a 1D vector so it can be used in the neural network, and normalize image values.

Convert the labels from numbers to one-hot-encoding

In [ ]:

from tensorflow.keras import utils
import random
import PML

#shuffle samples order
random.shuffle(samples)

#Split to input X and labels Y
X=np.array([i[0] for i in samples])
Y=np.array([i[1] for i in samples])
 
#Prepared to convolutional samples (3 channels per sample)
X=PML.prepare_conv_samples(X)
#normalize values
x_train,normalizer=PML.normalize_image(X)

#prepare labels to one-hot-encoding
nb_classes=len(labels)
y_train = utils.to_categorical(Y, nb_classes)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(x_train,y_train,test_size=0.1)

print("Training using: {0} samples".format(len(X_train)))
print("Validating using: {0} samples".format(len(X_test)))

## Model Creation

Create a model for training

In [ ]:

from tensorflow.keras.callbacks import TensorBoard
from time import time

input_shape=(TARGET_WIDTH,TARGET_HEIGHT,1)

model=PML.create_conv_classify_model(input_shape,nb_classes,32,[32,64,64],[64,32],dropout=0)
history=model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=20,batch_size=128)

#export model
PML.export_model(model,"./models/catsdogs")

In [19]:
from tensorflow.keras.utils import plot_model

plot_model(model,to_file="./models/catsdogs.png")


ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [ ]:
PML.plot_acc_loss(history)

## Test Samples

Load newly unseen samples to test them against the trained model. Check the accuracy of training.

In [ ]:
_=PML.test_samples(model,samples=sample_test,labels=labels,normalizer=normalizer,flatten=False)

## [Optional] OSC for realtime recognition


You can use this to communicate with external application (Processing for example) to predict images

In [17]:
#load model
TARGET_WIDTH=150
TARGET_HEIGHT=150
model=PML.import_model("./models/catsdogs")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [ ]:
#setup OSC Protocol to communicate with Processing
import OSCHelper
server=OSCHelper.createServer(9000)
client=OSCHelper.createClient(4200)

In [ ]:
import numpy as np
import io

def onOSC_Image(*args):
    width=args[1]
    height=args[2]
    jpgdata=args[3]
    file_jpgdata = io.BytesIO(jpgdata)
    img = Image.open(file_jpgdata).resize((TARGET_WIDTH,TARGET_HEIGHT)).convert("L")
    img=np.array(img).astype(float)
    img=img/255.0
    x=PML.prepare_conv_samples([x])
    res=model.predict(np.array(x))
    label_idx=np.argmax(res)
    client.send_message("/output/label",[int(label_idx),str(labels[label_idx])])
    #plt.imshow(img,cmap='gray')
    #plt.title("Prediction:{0}".format(labels[label_idx]))
    #plt.show()

server.addMsgHandler( "/inputs/image", onOSC_Image )

OSCHelper.start_server(server)


In [ ]:
server.close()
